In [6]:
import os
import sys
import numpy as np
import pandas as pd

notebook_name = 'aggeregate_metrics.ipynb'

current_dir = os.path.dirname(os.path.abspath(notebook_name))
parent_dir = os.path.abspath(os.path.join(current_dir, '..', '..', '..'))


sys.path.insert(1, parent_dir)
from src.data_processer import TelecomDataProcessor


In [2]:
# Replace 'your_conn_params' with your actual connection parameters
conn_params = {
    'host': 'localhost',
    'port': 5432,
    'user': 'ekru',
    'password': 'ekram12345',
    'database': 'tellco',
}

telecom_data_processor = TelecomDataProcessor(conn_params)


In [3]:
df = telecom_data_processor.load_dataset()

/home/ekram/Desktop/week-1/user_analytics_in_the_telecommunication_industry/src/data_processer.py:63: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


In [7]:
cleaned_df = telecom_data_processor.clean_dataset(df)

In [8]:
telecom_data_processor.overview_analysis(cleaned_df)

          BEARER_ID       START_MS         END_MS     DUR._(MS)          IMSI  \
count  1.500010e+05  150001.000000  150001.000000  1.500010e+05  1.500010e+05   
mean   1.013887e+19     499.661213     499.273526  1.046086e+05  2.082016e+14   
std    2.883600e+18     288.201515     287.687241  8.103735e+04  2.144722e+10   
min    6.917538e+18       1.000000       1.000000  7.142000e+03  2.040471e+14   
25%    7.349883e+18     251.000000     252.000000  5.744200e+04  2.082014e+14   
50%    7.349883e+18     499.661213     500.000000  8.639900e+04  2.082015e+14   
75%    1.304243e+19     749.000000     750.000000  1.324300e+05  2.082018e+14   
max    1.318654e+19     999.000000     999.000000  1.859336e+06  2.140743e+14   

       MSISDN/NUMBER          IMEI  AVG_RTT_DL_(MS)  AVG_RTT_UL_(MS)  \
count   1.500010e+05  1.500010e+05    150001.000000    150001.000000   
mean    4.188282e+10  4.847455e+13       109.803795        18.280156   
std     2.438731e+12  2.237359e+13       559.341973   

In [10]:
# Aggregate sessions frequency per customer
sessions_frequency = cleaned_df.groupby('MSISDN/NUMBER')['BEARER_ID'].count().reset_index()
sessions_frequency.columns = ['MSISDN/NUMBER', 'SESSIONS FREQUENCY']

# Aggregate session duration per customer
session_duration = cleaned_df.groupby('MSISDN/NUMBER')['DUR._(MS)'].sum().reset_index()
session_duration.columns = ['MSISDN/NUMBER', 'TOTAL SESSION DURATION (MS)']

# Aggregate total traffic per customer (download + upload)
total_traffic = cleaned_df.groupby('MSISDN/NUMBER')['TOTAL_UL_(BYTES)', 'TOTAL_DL_(BYTES)'].sum().reset_index()
total_traffic['TOTAL TRAFFIC (BYTES)'] = total_traffic['TOTAL_UL_(BYTES)'] + total_traffic['TOTAL_DL_(BYTES)']
total_traffic = total_traffic[['MSISDN/NUMBER', 'TOTAL TRAFFIC (BYTES)']]

# Report the top 10 customers for each metric
top_10_sessions_frequency = sessions_frequency.nlargest(10, 'SESSIONS FREQUENCY')
top_10_session_duration = session_duration.nlargest(10, 'TOTAL SESSION DURATION (MS)')
top_10_total_traffic = total_traffic.nlargest(10, 'TOTAL TRAFFIC (BYTES)')

# Display the results
print("Top 10 Customers by Sessions Frequency:")
print(top_10_sessions_frequency)

print("\nTop 10 Customers by Total Session Duration:")
print(top_10_session_duration)

print("\nTop 10 Customers by Total Traffic:")
print(top_10_total_traffic)


Top 10 Customers by Sessions Frequency:
        MSISDN/NUMBER  SESSIONS FREQUENCY
106853   4.188282e+10                1066
13526    3.362632e+10                  18
6437     3.361489e+10                  17
13180    3.362578e+10                  17
37052    3.365973e+10                  16
76363    3.367588e+10                  15
92923    3.376054e+10                  15
65118    3.366716e+10                  13
666      3.360313e+10                  12
1279     3.360452e+10                  12

Top 10 Customers by Total Session Duration:
        MSISDN/NUMBER  TOTAL SESSION DURATION (MS)
106853   4.188282e+10                 7.255100e+07
13180    3.362578e+10                 1.855375e+07
6437     3.361489e+10                 9.966898e+06
92923    3.376054e+10                 9.279434e+06
13526    3.362632e+10                 8.791927e+06
65118    3.366716e+10                 8.744914e+06
50281    3.366284e+10                 6.614270e+06
57160    3.366469e+10                 6.28873

/tmp/ipykernel_10537/3491706810.py:10: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  total_traffic = cleaned_df.groupby('MSISDN/NUMBER')['TOTAL_UL_(BYTES)', 'TOTAL_DL_(BYTES)'].sum().reset_index()
